In [344]:
'''This code is closely adapted
from google ml crash course'''


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
import numpy as np

In [345]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
y = y.reshape(150,1)
Data = np.concatenate((X,y), axis = 1)


In [346]:
def split_data(n_train):
    if (n_train < 0) or (n_train > 150):
        print ("Invalid number of training points")
        return
    np.random.seed(0)
    perm = np.random.permutation(150)
    training_indices = perm[range(0,n_train)]
    test_indices = perm[range(n_train,150)]
    trainx = Data[training_indices,:]
    trainy = y[training_indices]
    testx = Data[test_indices,:]
    testy = y[test_indices]
    return trainx, trainy, testx, testy


In [347]:
trainx, trainy,testx,testy = split_data(20)

In [348]:
list = []
for i in range(len(testy)):
    if testy[i] == 0:
        list.append('setosa')
        
    if testy[i] ==1:
        list.append('versicolor')
            
    if testy[i]==2:
        list.append('virginica')

In [349]:
def unique_vals(data, col):
    return set([row[col] for row in data])

In [350]:
header = ["Sepal_Length", "Sepal_Width", "Petal_Length" , "Petal_Width"]

In [351]:
def class_counts(data):
    counts = {}
    for i in range(len(data)):
        row = data[i]
        label = row[-1]
        
        if label == 0:
            label = 'setosa'
        if label ==1:
            label = 'versicolor'
            
        if label == 2:
            label = 'virginica'
            
            
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
        
    return counts

In [352]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value,float)

In [353]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        val = example[self.column]
        
        if is_numeric(val):
            return val >= self.value
        #else:
            #return val == self.value
        
        
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
            
        return  "Is %s %s %s?" % ( 
            header[self.column], condition, str(self.value))

In [354]:
def partition(data, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for i in range(len(data)):
        row = data[i]
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
            
    return true_rows, false_rows

In [355]:
def gini(data):
    
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(data)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(data))
        impurity -= prob_of_lbl**2
    return impurity
        

In [356]:
def info_gain(left, right, current_uncertainity):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    
    p = float(len(left)) / (len(left) + len(right))
    
    return current_uncertainity - p*gini(left) - (1-p)*gini(right)
    

In [357]:
def find_best_split(data):
    
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
        
    best_gain = 0
    best_question = None
    current_uncertainity = gini(data)
    
    n_features = len(data[0]) - 1 #number of columns
    
    for col in range(n_features):
        
        values = set([row[col] for row in data]) #unique values in the column
        
        for val in values: #for each values
            
            question = Question(col, val)
            
            #try splitting the datset
            
            true_rows, false_rows = partition(data, question)
            
            #skip this split if it doesnt divide the dataset
            
            if len(true_rows) == 0 or len(false_rows) ==0:
                continue
                
                
            gain = info_gain(true_rows, false_rows, current_uncertainity)
            
            if gain >= best_gain:
                best_gain, best_question = gain, question
    
    
    
    return best_gain, best_question
        
               
        
    


In [358]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """
    
    def __init__(self, rows):
        self.predictions = class_counts(rows)
    

In [359]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [360]:
def build_tree(data):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(data)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(data)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(data, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)





In [361]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [362]:
my_tree =build_tree(trainx)


In [363]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [364]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [365]:

predictions = []

for i in range(len(testx)):
    row = testx[i]
    #print ("Actual: %s. Predicted: %s" %
    #print(list[i],':', print_leaf(classify(row, my_tree)))
    
    predictions.append(classify(row, my_tree).keys())

virginica : {'versicolor': '100%'}
versicolor : {'versicolor': '100%'}
setosa : {'setosa': '100%'}
setosa : {'setosa': '100%'}
virginica : {'versicolor': '100%'}
setosa : {'setosa': '100%'}
setosa : {'setosa': '100%'}
versicolor : {'versicolor': '100%'}
versicolor : {'versicolor': '100%'}
setosa : {'setosa': '100%'}
virginica : {'virginica': '100%'}
versicolor : {'versicolor': '100%'}
setosa : {'setosa': '100%'}
virginica : {'versicolor': '100%'}
virginica : {'virginica': '100%'}
versicolor : {'versicolor': '100%'}
setosa : {'setosa': '100%'}
versicolor : {'virginica': '100%'}
versicolor : {'versicolor': '100%'}
versicolor : {'versicolor': '100%'}
virginica : {'virginica': '100%'}
setosa : {'setosa': '100%'}
virginica : {'virginica': '100%'}
setosa : {'setosa': '100%'}
setosa : {'setosa': '100%'}
versicolor : {'versicolor': '100%'}
virginica : {'virginica': '100%'}
virginica : {'virginica': '100%'}
virginica : {'versicolor': '100%'}
virginica : {'virginica': '100%'}
versicolor : {'vers